# Características del GPU

El objetivo del ejemplo es mostrar las caracteristicas del GPU que disponibiliza colab.

## Listar el tipo de GPU que esta activa en el cuaderno:

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-c11562c6-854e-2e84-2776-9ecf629761af)


## Listar máxima cantidad de Grillas /Bloques soportados por el GPU:

In [2]:
!cd  /usr/local/cuda/samples/1_Utilities/deviceQuery/; make >/dev/null
!/usr/local/cuda/samples/1_Utilities/deviceQuery/deviceQuery | grep "Max dimension"

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
  Max dimension size of a thread block (x,y,z): (1024, 1024, 64)
  Max dimension size of a grid size    (x,y,z): (2147483647, 65535, 65535)


# Ejemplo Hola Mundo con GPU.

## Se instala el módulo de cuda para python

In [4]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 7.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 69 kB 7.5 MB/s 
     |████████████████████████████████| 78 kB 7.7 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=626634 sha256=156708dbf7d7d150dbdef5b0f27c80da220fef10975fa43642c16298a1569d8b
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2022.1.9-py2.py3-none-any.whl size=64666 sha256=85ade819a8f30b87d253e74a44b59f4a09e5933991dc7dcda50db3d62597b64a
  Stored in directory: /root/.cache/pip/wheels/11/d8/85/01a8cf50977badf2da880865124b159b112b861655a9b631d3
Successfully built pycuda pytools


## Reinicio el buffer de plataforma Colab, donde la GPU escribe en lugar de la consola.

In [14]:
!>/var/colab/app.log

## Ejecuto el ejemplo Hola Mundo

Se puede demotrar el comportamiento de la forma de planificación de hilos. Ademas que el kernel ahora soporta la función printf().

In [15]:
#!/usr/bin/env python
# --------------------------------------------
#@title Parámetros de ejecución { vertical-output: true }

cantidad_N =   15#@param {type: "number"}
# --------------------------------------------
import numpy
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

pycuda.tools.make_default_context()

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
#include <stdio.h>
__global__ void kernel_HolaMundo( int N )
{
  int idx = threadIdx.x + blockIdx.x*blockDim.x;

  if( idx < N )
  {
    printf( "Hola Mundo desde el GPU - idx %d, Bloque id %d, Thread id %d \\n ", idx, blockIdx.x, threadIdx.x );
  }
  else
  {
    printf( "No saludo, porque soy un hilo planificado de mas - idx %d \\n ", idx );
  }

}
""")

# CPU - Genero la función kernel.
kernel = module.get_function("kernel_HolaMundo")

dim_hilo = 32
dim_bloque = int( (cantidad_N+dim_hilo-1) / dim_hilo )

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel( numpy.int32(cantidad_N),  block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

cuda.Context.synchronize()

print( "Hola Mundo desde el CPU => Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )



Hola Mundo desde el CPU => Thread x:  32 , Bloque x: 1


Muestro el buffer de COLAB: 

In [16]:
cat /var/colab/app.log

{"pid":7,"type":"jupyter","level":40,"msg":"Hola Mundo desde el GPU - idx 0, Bloque id 0, Thread id 0 ","time":"2022-06-06T17:22:40.810Z","v":0}
{"pid":7,"type":"jupyter","level":40,"msg":" Hola Mundo desde el GPU - idx 1, Bloque id 0, Thread id 1 ","time":"2022-06-06T17:22:40.810Z","v":0}
{"pid":7,"type":"jupyter","level":40,"msg":" Hola Mundo desde el GPU - idx 2, Bloque id 0, Thread id 2 ","time":"2022-06-06T17:22:40.810Z","v":0}
{"pid":7,"type":"jupyter","level":40,"msg":" Hola Mundo desde el GPU - idx 3, Bloque id 0, Thread id 3 ","time":"2022-06-06T17:22:40.811Z","v":0}
{"pid":7,"type":"jupyter","level":40,"msg":" Hola Mundo desde el GPU - idx 4, Bloque id 0, Thread id 4 ","time":"2022-06-06T17:22:40.811Z","v":0}
{"pid":7,"type":"jupyter","level":40,"msg":" Hola Mundo desde el GPU - idx 5, Bloque id 0, Thread id 5 ","time":"2022-06-06T17:22:40.811Z","v":0}
{"pid":7,"type":"jupyter","level":40,"msg":" Hola Mundo desde el GPU - idx 6, Bloque id 0, Thread id 6 ","time":"2022-06-06T1

# Profiler en cuda.

In [ ]:
!cd /usr/local/cuda/samples/0_Simple/vectorAdd/; make >/dev/null
#!/usr/local/cuda/samples/bin/x86_64/linux/release/vectorAdd
!/usr/local/cuda/bin/nvprof --csv --concurrent-kernels on --openmp-profiling on --print-gpu-trace --normalized-time-unit us --print-gpu-trace /usr/local/cuda/samples/bin/x86_64/linux/release/vectorAdd

[Vector addition of 50000 elements]
==346== NVPROF is profiling process 346, command: /usr/local/cuda/samples/bin/x86_64/linux/release/vectorAdd
==346== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
Copy input data from the host memory to the CUDA device
CUDA kernel launch with 196 blocks of 256 threads
Copy output data from the CUDA device to the host memory
Test PASSED
Done
==346== Profiling application: /usr/local/cuda/samples/bin/x86_64/linux/release/vectorAdd
==346== Profiling result:
"Start","Duration","Grid X","Grid Y","Grid Z","Block X","Block Y","Block Z","Registers Per Thread","Static SMem","Dynamic SMem","Size","Throughput","SrcMemType","DstMemType","Device","Context","Stream","Name","Correlation_ID"
us,us,,,,,,,,B,B,KB,GB/s,,,,,,,
356140.739000,38.911000,,,,,,,,,,195.312500,4.786937,"Pageable","Device","Tesla K80 (0)","1","7","[CUDA memcpy HtoD]",114
356220.130000,28.767000,,,,,,,,,,195.312500,6.474937,"Pageable","Device","Tesla K80 (0)","1

# Debug con  CUDA.

Ver el ejemplo desde: 
https://wiki.tiker.net/PyCuda/FrequentlyAskedQuestions/#system-specific-questions


In [ ]:
!cuda-gdb 

NVIDIA (R) CUDA Debugger
11.0 release
Portions Copyright (C) 2007-2020 NVIDIA Corporation
GNU gdb (GDB) 8.2
Copyright (C) 2018 Free Software Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.
Type "show copying" and "show warranty" for details.
This GDB was configured as "x86_64-pc-linux-gnu".
Type "show configuration" for configuration details.
For bug reporting instructions, please see:
<http://www.gnu.org/software/gdb/bugs/>.
Find the GDB manual and other documentation resources online at:
    <http://www.gnu.org/software/gdb/documentation/>.

For help, type "help".
Type "apropos word" to search for commands related to "word".
(cuda-gdb) help
List of classes of commands:

aliases -- Aliases of other commands
breakpoints -- Making program stop at certain points
cuda  -- CUDA commands
data -- Examining data
files -- Specif